In [ ]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import logging
from joblib import Parallel, delayed
import multiprocessing
import json
from datetime import datetime
import os

In [ ]:
class MonteCarloSupplyChainOptimizer:
    def __init__(self, demand_df, prices_df, max_load=8000, transport_cost=4500, iterations=1000, n_jobs=-1):
        self.original_demand_df = demand_df.copy()
        self.prices_df = prices_df.copy()
        self.max_load = max_load
        self.transport_cost = transport_cost
        self.iterations = iterations
        self.n_jobs = n_jobs if n_jobs != -1 else multiprocessing.cpu_count()

        self.demands = self.original_demand_df.groupby(['polygon', 'specie'])['demand'].sum().reset_index()
        self.species = self.demands['specie'].unique()
        self.suppliers = self.prices_df['supplier'].unique()

        self.log_file = 'MC_Supply_Chain.log'
        if os.path.exists(self.log_file):
            logging.shutdown()
            os.remove(self.log_file)

    def generate_random_solution(self):
        remaining = self.demands.sample(frac=1).copy()
        schedule = []
        day = 0

        while remaining['demand'].sum() > 0:
            day += 1
            remaining_capacity = self.max_load
            day_plan = []
            fulfilled_indices = []

            for idx, row in remaining[remaining['demand'] > 0].iterrows():
                polygon, specie, demand = row['polygon'], row['specie'], row['demand']
                if demand > remaining_capacity:
                    continue

                suppliers = self.prices_df[self.prices_df['specie'] == specie]['supplier'].tolist()
                if not suppliers:
                    continue

                supplier = random.choice(suppliers)

                day_plan.append((day, polygon, specie, supplier, demand))
                remaining_capacity -= demand
                fulfilled_indices.append(idx)

            schedule.extend(day_plan)
            remaining.loc[fulfilled_indices, 'demand'] = 0

        return schedule

    def fitness(self, schedule):
        cost = 0

        horizon_days = max(day_plan[0] for day_plan in schedule)
        for day in range(1, horizon_days + 1):
            day_orders = [entry for entry in schedule if entry[0] == day]
            day_suppliers = set()
            day_load = 0

            for (_, polygon, specie, supplier, amount) in day_orders:
                row = self.prices_df[(self.prices_df['specie'] == specie) & (self.prices_df['supplier'] == supplier)]

                unit_price = row.iloc[0]['price']
                cost += unit_price * amount
                day_load += amount
                day_suppliers.add(supplier)

            cost += self.transport_cost * len(day_suppliers)

        return cost

    def evaluate_one_solution(self, i):
        candidate = self.generate_random_solution()
        candidate_cost = self.fitness(candidate)
        return candidate, candidate_cost

    def run(self):
        logging.info("Starting parallel Monte Carlo optimization...")

        results = Parallel(n_jobs=self.n_jobs)(
            delayed(self.evaluate_one_solution)(i) for i in tqdm(range(self.iterations), desc='Finding solutions...')
        )

        best_solution, best_cost = min(results, key=lambda x: x[1])
        logging.info(f"Best solution found with cost: {best_cost:.2f}")
        
        return best_solution, best_cost

In [ ]:
demand_df = pd.read_csv('../setup/demand.csv')
prices_df = pd.read_csv('../setup/supplier_prices.csv')

In [ ]:
demand_df

In [ ]:
optimizer = MonteCarloSupplyChainOptimizer(demand_df, prices_df, iterations=10_000)
best_solution, best_cost = optimizer.run()

In [ ]:
temp = pd.DataFrame(best_solution, columns = ['Day', 'Polygon', 'Specie', 'Supplier', 'Amount'])

orders = temp.groupby(['Day', 'Supplier', 'Specie']).agg({'Amount' : 'sum'}).reset_index()
vrp_nodes = temp.groupby('Day').agg({'Amount' : 'unique'}).reset_index()

orders_list = orders.to_dict(orient='records')
with open('../scheduling/MC_orders.json', 'w') as f:
    json.dump(orders_list, f, indent=2)

In [ ]:
filepath = '../scheduling/MC_Supply_Chain_Solution.json'

# Convert numpy.float64 to float for JSON serialization
save_data = {
    'best_solution': best_solution,
    'best_cost': float(best_cost),
    'datetime': datetime.now().isoformat()
}

if not os.path.exists(filepath):
    with open(filepath, 'w') as f:
        json.dump(save_data, f, indent=2)
    print("Solution saved.")
else:
    with open(filepath, 'r') as f:
        existing_data = json.load(f)
    if float(best_cost) < float(existing_data['best_cost']):
        with open(filepath, 'w') as f:
            json.dump(save_data, f, indent=2)
        print("New best solution saved.")
    else:
        print("Existing solution is better or equal. No changes made.")